# Find the Protein Structure with the Minimum Energy

<ol>

<li>First, use the <b> ProteinFoldingProblem </b> approach outlined in: <br><br>
npj Quantum Information 7 (2021), <i> Resource-efficient quantum algorithm for protein folding </i><br>
by Anton Robert, Panagiotis Kl. Barkoutsos, Stefan Woerner & Ivano Tavernelli <br>
<br></li>

<li>Next, find the minimum energy using the <b>iQPE algorithm </b>. <br></li>

</ol>

## ProteinFoldingProblem approach

In [1]:
from protein_folding.protein_folding_problem import ProteinFoldingProblem
from protein_folding.peptide.peptide import Peptide
from protein_folding.interactions.random_interaction import RandomInteraction
from protein_folding.interactions.miyazawa_jernigan_interaction import MiyazawaJerniganInteraction
from protein_folding.penalty_parameters import PenaltyParameters
from qiskit.utils import algorithm_globals, QuantumInstance
algorithm_globals.random_seed = 23

ImportError: Qiskit is installed in an invalid environment that has both Qiskit >=1.0 and an earlier version. You should create a new virtual environment, and ensure that you do not mix dependencies between Qiskit <1.0 and >=1.0. Any packages that depend on 'qiskit-terra' are not compatible with Qiskit 1.0 and will need to be updated. Qiskit unfortunately cannot enforce this requirement during environment resolution. See https://qisk.it/packaging-1-0 for more detail.

### Define a ProteinFoldingProblem

#### Focus on a short peptide chain
#### Don't include any side chains

In [25]:
# qiskit_research tutorial chain
# main_chain = "APRLRFY" 
# main_chain = "APRLR" 
# fractal analytics Alzheimer's enzyme related chain
main_chain = "YPYFIP"
main_chain_len = len(main_chain)
print("main_chain_len", main_chain_len)
side_chains = [""] * main_chain_len
peptide = Peptide(main_chain, side_chains)

main_chain_len 6


#### Choose interaction type
#### Assign structure penalties for folding back on itself and chirality

In [19]:
random_interaction = RandomInteraction()
mj_interaction = MiyazawaJerniganInteraction()
penalty_back = 10
penalty_chiral = 10
penalty_1 = 10
penalty_terms = PenaltyParameters(penalty_chiral, penalty_back, penalty_1)

#### Create the ProteinFoldingProblem

In [ ]:
protein_folding_problem = ProteinFoldingProblem(peptide, mj_interaction, penalty_terms)

#### Get the Hamiltonian by calling the qubit_op method

In [21]:
qubit_op = protein_folding_problem.qubit_op()
print(type(qubit_op))
print(qubit_op)

<class 'qiskit.opflow.primitive_ops.pauli_sum_op.PauliSumOp'>
929.4905 * IIIIII
+ 300.0 * IIIZII
- 97.5 * IIIIZZ
+ 97.5 * IIIZZZ
- 100.0 * IZIZII
- 100.0 * IIZIZI
- 100.0 * IZZZZI
+ 202.5 * IIIZZI
- 310.0 * IZIIII
- 207.5 * IZZIII
+ 205.0 * IIIIIZ
+ 102.5 * IIZIIZ
- 102.5 * IZZIIZ
- 924.4905 * ZIIIII
- 302.5 * ZIIZII
- 202.5 * ZIIZZI
+ 310.0 * ZZIIII
+ 207.5 * ZZZIII
+ 102.5 * ZZIZII
+ 102.5 * ZIZIZI
+ 102.5 * ZZZZZI
- 205.0 * ZIIIIZ
+ 100.0 * ZIIIZZ
- 100.0 * ZIIZZZ
- 102.5 * ZIZIIZ
+ 102.5 * ZZZIIZ
- 2.5 * IIIIZI
+ 2.5 * IIZIII
+ 2.5 * ZIIIZI
- 2.5 * ZIZIII


#### Verify the number of qubits used, and check the size of the operator matrix

In [4]:
print(qubit_op.num_qubits, 2**qubit_op.num_qubits)

6 64


#### Find the minimum energy and structure for that energy

In [8]:
from qiskit.circuit.library import RealAmplitudes
from qiskit.algorithms.optimizers import COBYLA
from qiskit.algorithms import NumPyMinimumEigensolver
from qiskit.algorithms.minimum_eigensolvers import SamplingVQE
from qiskit import execute, Aer
from qiskit.primitives import Sampler

# set classical optimizer
optimizer = COBYLA(maxiter=50)

# set variational ansatz
ansatz = RealAmplitudes(reps=1)

counts = []
values = []


def store_intermediate_result(eval_count, parameters, mean, std):
    counts.append(eval_count)
    values.append(mean)


# initialize VQE using CVaR with alpha = 0.1
vqe = SamplingVQE(
    Sampler(),
    ansatz=ansatz,
    optimizer=optimizer,
    aggregation=0.1,
    callback=store_intermediate_result,
)

raw_result = vqe.compute_minimum_eigenvalue(qubit_op)
print(raw_result)

SamplingMinimumEigensolverResult:
	Eigenvalue: -1.0190000000001191
	Best measurement
: {'state': 37, 'bitstring': '100101', 'value': (-1.0190000000001191+0j), 'probability': 0.223520808846322}



## iQPE Approach

#### We need an ansatz, and a unitary operator
We can use the ansatz used in the ProteinFoldingProblem <br>
We can get a unitary by exponentiating the Hamiltonian (qubit_op) from the ProteinFoldingProblem <br>
We can create a TimeEvolutionProblem and evolve the Hamiltonian via TrotterQRTE to get a unitary as a circuit <br>

In [28]:
from qiskit import QuantumCircuit
from qiskit_algorithms import IterativePhaseEstimation

In [29]:
from qiskit_algorithms import TimeEvolutionProblem, TrotterQRTE
from qiskit import BasicAer
from qiskit.utils import QuantumInstance
from qiskit.primitives import Estimator

#### Define the TimeEvolutionProblem

In [15]:
hamiltonian = qubit_op
time = 1
initial_state = ansatz
#initial_state = None
time_evo_problem = TimeEvolutionProblem(hamiltonian, time, initial_state)

In [30]:
estimator = Estimator()
trotter_qrte = TrotterQRTE(estimator=estimator)
evolved_state = trotter_qrte.evolve(time_evo_problem).evolved_state

TypeError: TrotterQRTE.__init__() got an unexpected keyword argument 'estimator'

#### Define the TrotterQRTE

In [16]:
backend = BasicAer.get_backend("statevector_simulator")
quantum_instance = QuantumInstance(backend=backend)
trotter_qrte = TrotterQRTE(quantum_instance=quantum_instance)
evolved_state = trotter_qrte.evolve(time_evo_problem).evolved_state
print(type(evolved_state))

AttributeError: 'TimeEvolutionProblem' object has no attribute 'param_value_dict'

In [42]:
num_iterations = 3
quantum_instance = None
qc = QuantumCircuit()
sampler = Sampler()
ipe = IterativePhaseEstimation(num_iterations, quantum_instance, sampler)

In [44]:
import numpy as np
m = qubit_op_matrix
np.allclose(np.eye(len(m)), m.dot(m.T.conj()))

False

In [45]:
m = qubit_op_unitary
np.allclose(np.eye(len(m)), m.dot(m.T.conj()))

False

In [46]:
#qc_qubit_op = QuantumCircuit(3)
#qc_qubit_op.unitary(qubit_op_unitary, [0,1,2])
#print(type(qc_qubit_op))

ExtensionError: 'Input matrix is not unitary.'

In [ ]:
result = ipe.estimate(qc_qubit_op, ansatz)
result